This is an example of how to hit the OpenAire API

In [5]:
import requests

def query_openaire(datasets, result_limit=10):
    url = "https://api.openaire.eu/graph/researchProducts"
    headers = {"accept": "application/json"}
    results = {}
    
    for dataset in datasets:
        params = {
            "search": dataset,
            "type": "publication",
            "page": 1,
            "pageSize": result_limit,
            "sortBy": "relevance DESC"
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            formatted_results = []
            
            for item in data.get("results", []):
                formatted_results.append({
                    "Title": item.get("mainTitle", "N/A"),
                    "Publication Date": item.get("publicationDate", "N/A"),
                    "Authors": ", ".join([author.get("fullName", "N/A") for author in item.get("author", [])]),
                    "Subjects": ", ".join([subj.get("subject", {}).get("value", "N/A") for subj in item.get("subjects", [])]),
                    "Publisher": item.get("publisher", "N/A"),
                    "Access Right": item.get("bestAccessRight", {}).get("label", "N/A"),
                    "DOI": ", ".join([pid.get("value", "N/A") for pid in item.get("pid", [])]),
                    "URL": ", ".join([inst.get("url", ["N/A"])[0] for inst in item.get("instance", [])])
                })
            
            results[dataset] = formatted_results
        else:
            results[dataset] = {"error": f"Failed to retrieve data: {response.status_code}"}
    
    return results

# Example usage
dataset_names = ["UKBiobank", "Genomics England"]
query_results = query_openaire(dataset_names)
print(query_results)

{'UKBiobank': [{'Title': 'BioBankRead: Data pre-processing in Python for UKBiobank clinical data', 'Publication Date': '2019-04-26', 'Authors': 'Schneider-Luftman, D., Crum, W. R.', 'Subjects': '03 medical and health sciences, 0302 clinical medicine', 'Publisher': 'Cold Spring Harbor Laboratory', 'Access Right': 'OPEN', 'DOI': '10.1101/569715, 2940481546', 'URL': 'https://doi.org/10.1101/569715, https://www.biorxiv.org/content/biorxiv/early/2019/04/26/569715.full.pdf, https://www.biorxiv.org/content/10.1101/569715v1, https://doi.org/10.1101/569715'}, {'Title': 'Polygenic subtype identified in ACCORD trial displays a favorable type 2 diabetes phenotype in the UKBiobank population', 'Publication Date': '2024-06-22', 'Authors': 'Hershberger, Courtney, Pantalone, Kevin M., Mariam, Arshiya, Buse, John B., Motsinger-Reif, Alison A., Rotroff, Daniel M.', 'Subjects': 'Subtypes, Single-nucleotide polymorphism (SNP), Brief Report, Genetics, R, Medicine, Type 2 diabetes, Genomics, QH426-470, A1c 